# Tutorial: Using QSweeper

Scroll down to begin.

In [1]:
%reload_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\pyproj\__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


In [3]:
design = designs.DesignPlanar({}, True)
gui = MetalGUI(design)

In [4]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um',
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))

gui.rebuild()
gui.autoscale()

In [5]:
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='1.75mm',  pos_y='0um', orientation='0'))
RouteMeander(design, 'readout',  Dict(
        total_length='6 mm',
        hfss_wire_bonds = True,
        fillet='90 um',
        lead = dict(start_straight='100um'),
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='readout'),
            end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()

In [24]:
q1.options

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'connection_pads': {'readout': {'pad_gap': '15um',
   'pad_width': '200um',
   'pad_height': '30um',
   'pad_cpw_shift': '5um',
   'pad_cpw_extent': '25um',
   'cpw_width': 'cpw_width',
   'cpw_gap': 'cpw_gap',
   'cpw_extend': '100um',
   'pocket_extent': '5um',
   'pocket_rise': '65um',
   'loc_W': 1,
   'loc_H': 1}},
 'pad_gap': '30um',
 'inductor_width': '20um',
 'pad_width': '425 um',
 'pad_height': '90um',
 'pocket_width': '650um',
 'pocket_height': '650um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'hfss_inductance': '10nH',
 'hfss_capacitance': 0,
 'hfss_resistance': 0,
 'hfss_mesh_kw_jj': 7e-06,
 'q3d_inductance': '10nH',
 'q3d_capacitance': 0,
 'q3d_resistance': 0,
 'q3d_mesh_kw_jj': 7e-06,
 'gds_cell_name': 'my_other_junction'}

## Use EPRanalysis

In [6]:
from qiskit_metal.analyses.quantization import EPRanalysis

In [7]:
eig_qb = EPRanalysis(design, "hfss")

In [8]:
# example: update single setting
eig_qb.sim.setup.max_passes = 6
eig_qb.sim.setup.vars.Lj = '11 nH'
eig_qb.sim.setup.reuse_selected_design = False
# example: update multiple settings
eig_qb.sim.setup_update(max_delta_f = 0.4, min_freq_ghz = 1.1)

eig_qb.sim.setup.n_modes = 2

eig_qb.sim.setup

{'name': 'Setup',
 'reuse_selected_design': False,
 'reuse_setup': True,
 'min_freq_ghz': 1.1,
 'n_modes': 2,
 'max_delta_f': 0.4,
 'max_passes': 6,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '11 nH', 'Cj': '0 fF'}}

In [9]:
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'
eig_qb.setup

{'junctions': {'jj': {'Lj_variable': 'Lj',
   'Cj_variable': 'Cj',
   'rect': 'JJ_rect_Lj_Q1_rect_jj',
   'line': 'JJ_Lj_Q1_rect_jj_'}},
 'dissipatives': {'dielectrics_bulk': ['main']},
 'cos_trunc': 8,
 'fock_trunc': 7,
 'sweep_variable': 'Lj'}

# START OF TUTORIAL: Setup QSweeper

In [22]:
import sys; sys.path.append(r"D:/Clark")

In [23]:
from QubitPresimulated.sweeper import QSweeper

# It has one initalization input, which is a QAnalysis module
# In this case: type(eig_qb) is EPRanalysis
sweeper = QSweeper(eig_qb)

In [27]:
# Tell the program which component you want to sweep through
target_component = 'Q1'

# and which parameter to change. The format of this dict is the same as
# QComponent.options. So if you want to change claw_length:
# parameters_to_sweep = {'connection_pads':{'res_claw':{'claw_length':[1,2,3]}}}

# Basically, instead of having strings / floats as values
# you now have lists of values you want to sweep over.
parameters_to_sweep = {'pad_width': ['425 um'],
                       'pad_height': ['90um', '135um']}

# Anything after save_path is put in **kwargs which gets sent to
# analysis.run(**kwargs)
sweeper.run_sweep(component_name = target_component,
                  parameters = parameters_to_sweep,
                  data_name = None,
                  save_path = r'./finding_parameters.csv',
                  components=['Q1', 'readout', 'open_to_ground'],
                  open_terminations=[('readout', 'end')]) # REQUIRED: must declare components= for this to wor


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]INFO 04:37PM [connect_design]: 	Opened active design
	Design:    Design_hfss1 [Solution type: Eigenmode]
WARNING 04:37PM [connect_setup]: 	No design setup detected.
WARNING 04:37PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:37PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:37PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:37PM [analyze]: Analyzing setup Setup
  df = pd.read_csv(io.StringIO(text2[3].strip()),

04:38PM 00s INFO [get_f_convergence]: Saved convergences to D:\Clark\QubitPresimulated\testing\hfss_eig_f_convergence.csv


Design "Design_hfss1" info:
	# eigenmodes    2
	# variations    1
Design "Design_hfss1" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 7.3653550535357e-24
        energy_elec_substrate = 6.7100334870998e-24
        EPR of substrate = 91.1%

        energy_mag    = 6.64713632452063e-26
        energy_mag % of energy_elec_all  = 0.9%
        

Variation 0  [1/1]

  Mode 0 at 6.05 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.1%  3.683e-24 3.324e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 96.92%
	jj              0.900578  (+)        0.0286249
		(U_tot_cap-U_tot_ind)/mean=6.14%
Calculating Qdielectric_main for mode 0 (0/1)
p_dielectric_main_0 = 0.9110264798271583

  Mode 1 at 8.89 GHz   [2/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_

  df = df.drop('Unnamed: 9', 1)



p_dielectric_main_1 = 0.9045205981557303


WARNING 04:38PM [__init__]: <p>Error: <class 'IndexError'></p>


...

WARNING 04:39PM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project45\Design_hfss2\2023-01-11 16-39-18.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization
Pm_norm=
modes
0    1.127302
1    1.085079
dtype: float64

Pm_norm idx =
      jj
0   True
1  False
*** P (participation matrix, not normlz.)
         jj
0  0.879353
1  0.002028

*** S (sign-bit matrix)
   s_jj
0     1
1     1
*** P (participation matrix, normalized.)
      0.99
     0.002

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       262     1.73
      1.73  0.00285

*** Chi matrix ND (MHz) 
       291     1.55
      1.55  0.00225

*** Frequencies O1 PT (MHz)
0    5363.041775
1    9067.311583
dtype: float64

*** Frequencies ND (MHz)
0    5349.299028
1    9067.330974
dtype: float64

*** Q_coupling
Empty DataFrame
Columns: []
Index: [0, 1]


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:30<00:00, 75.27s/it]

Simulated and logged configuration: {'pad_width': '425 um', 'pad_height': '135um'}


Traceback (most recent call last):
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\backends\backend_qt.py", line 455, in _draw_idle
    self.draw()
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\backends\backend_agg.py", line 436, in draw
    self.figure.draw(self.renderer)
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\artist.py", line 73, in draw_wrapper
    result = draw(artist, renderer, *args, **kwargs)
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\artist.py", line 50, in draw_wrapper
    return draw(artist, renderer)
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\figure.py", line 2810, in draw
    mimage._draw_list_compositing_images(
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\image.py", line 132, in _draw_list_compositing_images
    a.draw(renderer)
  File "D:\Users\lfl\Anaconda3\envs\qiskit-

In [32]:
sweeper.librarian.qoptions['pad_height']

0     90um
1    135um
Name: pad_height, dtype: object

In [37]:
sims = sweeper.librarian.simulations
sims['f_0']

0    0    6049.509517
1    8887.073683
Name: 0, dty...
1    0    5625.490630
1    9068.177289
Name: 0, dty...
Name: f_0, dtype: object

# Start here

In [11]:
ansys = eig_qb.sim.renderer

In [12]:
eig_qb.run_epr()

Design "Qbit_hfss3" info:
	# eigenmodes    2
	# variations    1
Design "Qbit_hfss3" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 7.3653550535357e-24
        energy_elec_substrate = 6.7100334870998e-24
        EPR of substrate = 91.1%

        energy_mag    = 6.64713632452063e-26
        energy_mag % of energy_elec_all  = 0.9%
        

Variation 0  [1/1]

  Mode 0 at 6.05 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.1%  3.683e-24 3.324e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 96.92%
	jj              0.900578  (+)        0.0286249
		(U_tot_cap-U_tot_ind)/mean=6.14%
Calculating Qdielectric_main for mode 0 (0/1)
p_dielectric_main_0 = 0.9110264798271583

  Mode 1 at 8.89 GHz   [2/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E   

  df = df.drop('Unnamed: 9', 1)



p_dielectric_main_1 = 0.9045205981557303


WARNING 04:18PM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project45\Qbit_hfss3\2023-01-11 16-18-19.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization
Pm_norm=
modes
0    1.131546
1    1.045867
dtype: float64

Pm_norm idx =
      jj
0   True
1  False
*** P (participation matrix, not normlz.)
         jj
0  0.875517
1  0.003055

*** S (sign-bit matrix)
   s_jj
0     1
1     1
*** P (participation matrix, normalized.)
      0.99
    0.0031

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       302     2.74
      2.74   0.0062

*** Chi matrix ND (MHz) 
       339     2.27
      2.27  0.00432

*** Frequencies O1 PT (MHz)
0    5746.006056
1    8885.698915
dtype: float64

*** Frequencies ND (MHz)
0    5728.670118
1    8885.765618
dtype: float64

*** Q_coupling
Empty DataFrame
Columns: []
Index: [0, 1]


ValueError: Data has no positive values, and therefore can not be log-scaled.

Traceback (most recent call last):
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\backends\backend_qt.py", line 455, in _draw_idle
    self.draw()
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\backends\backend_agg.py", line 436, in draw
    self.figure.draw(self.renderer)
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\artist.py", line 73, in draw_wrapper
    result = draw(artist, renderer, *args, **kwargs)
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\artist.py", line 50, in draw_wrapper
    return draw(artist, renderer)
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\figure.py", line 2810, in draw
    mimage._draw_list_compositing_images(
  File "D:\Users\lfl\Anaconda3\envs\qiskit-metal\lib\site-packages\matplotlib\image.py", line 132, in _draw_list_compositing_images
    a.draw(renderer)
  File "D:\Users\lfl\Anaconda3\envs\qiskit-

## Found $\omega_q$

In [18]:
def find_qubit_properties(EPRanalysis):
    '''
    From a runned EPR analysis, returns parameters of interest
    
    Input:
    * EPRanalysis (EPRanalysis)
    
    Output:
    * f_0 (float) - Eigenmode frequencies computed by HFSS
    * f_1 (float) - Dressed mode frequencies (based on 1st order perturbation theory on 4th order expansion of cosine)
    * chi_ND (pd.DataFrame) - chi matrix. On diagonals are anharmonicities. Off diagonals are dispersive shifts
    '''
    results = eig_qb.sim.renderer.epr_quantum_analysis.results
    
    return results

results = find_qubit_properties(eig_qb)
results
    

HamiltonianResultsContainer([('0',
                              OrderedDict([('f_0', 0    6049.509517
                                            1    8887.073683
                                            Name: 0, dtype: float64),
                                           ('f_1',
                                            0    5746.006056
                                            1    8885.698915
                                            dtype: float64),
                                           ('f_ND',
                                            0    5728.670118
                                            1    8885.765618
                                            dtype: float64),
                                           ('chi_O1',
                                                        0         1
                                            0  302.134893  2.737137
                                            1    2.737137  0.006199),
                                     

In [ ]:
results['chi_ND']

## Found 

In [ ]:
eig_qb.energy_elec